<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/Silueta_bootstrapV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade kmodes

In [2]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [3]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/df042421.csv")

In [6]:
data1['AISLE']=data1['AISLE'].astype(str)

In [7]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [8]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455


In [9]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [10]:
# Matriz de distancias
dm = pd.read_csv("/content/gdrive/MyDrive/TDGdata/distancias.csv")
dm=dm.to_numpy()

In [11]:
import time
start_time = time.time()
# Coeficiente silueta para K 2-4
kValues = [2, 3, 4]
gamma = 0.5
nBootstrap = 10
# Inicializamos la matriz de coeficientes silueta con ceros
silueta = [ [ 0 for j in range(nBootstrap + 1) ] for i in range(len(kValues)) ]
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]
numColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('float64').columns)]
N = data1.shape[0]

In [12]:
from sklearn.metrics import silhouette_score
for k in kValues:
  print('k = ', k)
  kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
  kprototype.fit_predict(data1, categorical = catColumnsPos)
  print("--- %s seconds ---" % (time.time() - start_time))
  silueta[k-min(kValues)][0] = silhouette_score(dm, kprototype.labels_, metric='precomputed')
  print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
  print(silueta[k-min(kValues)][0])
  for b in range(1,nBootstrap+1):
    print('bootstrap # ',b);
    boot = resample(data1, replace=True, n_samples=data1.shape[0], random_state=b)
    # Fit the cluster
    kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 1)
    kprototypebs.fit_predict(boot, categorical = catColumnsPos)
    print("--- %s seconds ---" % (time.time() - start_time));
    # boot dividido en atributos numericas y categoricos
    dataNum=boot.iloc[:,numColumnsPos]
    dataNum=dataNum.to_numpy()
    dataCat=boot.iloc[:,catColumnsPos]
    dataCat=dataCat.to_numpy()
    # Inicializamos la matriz de distancias con ceros
    scores = [ [ 0 for i in range(N) ] for j in range(N) ]
    # Calculamos la mitad de la matriz distancias
    for i in range(N):
      for j in range(i+1,N):
        scores[i][j]=np.sum((dataNum[i] - dataNum[j]) ** 2) + gamma * np.sum(dataCat[i]!=dataCat[j])
    print("Matriz de distancias --- %s seconds ---" % (time.time() - start_time))
    # Completamos la matriz
    for i in range(0,N):
      for j in range(0,i):
        scores[i][j]=scores[j][i]
    print("Espejo --- %s seconds ---" % (time.time() - start_time))
    silueta[k-min(kValues)][b] = silhouette_score(scores, kprototypebs.labels_, metric='precomputed')
    print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
    print(silueta[k-min(kValues)][b])

k =  2
--- 59.54181385040283 seconds ---
Coeficiente silueta --- 62.17752385139465 seconds ---
0.736586459830793
bootstrap #  1
--- 104.86763668060303 seconds ---
Matriz de distancias --- 2309.3918855190277 seconds ---
Espejo --- 2364.8357417583466 seconds ---
Coeficiente silueta --- 2406.583316087723 seconds ---
0.8101794559566166
bootstrap #  2
--- 2456.6112143993378 seconds ---
Matriz de distancias --- 4759.116342782974 seconds ---
Espejo --- 4810.561563968658 seconds ---
Coeficiente silueta --- 4853.7597506046295 seconds ---
0.7336153098017565
bootstrap #  3
--- 4903.415411949158 seconds ---
Matriz de distancias --- 7182.508513450623 seconds ---
Espejo --- 7236.964097499847 seconds ---
Coeficiente silueta --- 7279.135075092316 seconds ---
0.6896583725738442
bootstrap #  4
--- 7326.439989328384 seconds ---
Matriz de distancias --- 9576.024886131287 seconds ---
Espejo --- 9628.06230354309 seconds ---
Coeficiente silueta --- 9670.564180135727 seconds ---
0.7460526246725642
bootstrap #

In [13]:
data1['Cluster'] = kprototype.labels_

In [14]:
silueta

[[0.736586459830793,
  0.8101794559566166,
  0.7336153098017565,
  0.6896583725738442,
  0.7460526246725642,
  0.7267804280793367,
  0.7533110239832822,
  0.7257894086707366,
  0.7523972366754391,
  0.7629930334233889,
  0.7255889676697296],
 [0.7182138218409522,
  0.09330388061499972,
  0.0795551613919883,
  0.08579445355521252,
  0.7848836886437416,
  0.7580501112931499,
  0.09604875554522227,
  0.08272228147006884,
  0.08775924380328577,
  0.1008091939490596,
  0.10999866521757545],
 [0.7507712291842021,
  0.123667752437498,
  0.7566801991889119,
  0.7449492595601183,
  0.773342891525885,
  0.7369201592353902,
  0.7501565513476068,
  0.7134535728495904,
  0.7723386443651569,
  0.7503756482354623,
  0.7426714582364835]]

In [15]:
for i in kValues:
  print('Mediana para k = ',i);
  print(median(silueta[i-min(kValues)][1:nBootstrap+1]));

Mediana para k =  2
0.7398339672371603
Mediana para k =  3
0.094676318080111
Mediana para k =  4
0.7475529054538625


In [16]:
#np.savetxt("/content/gdrive/MyDrive/TDGdata/siluetaboot.csv", silueta, delimiter=",")